In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# %load_ext autotime  # must be added to dependencies
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from structure_factor.structure_factor_new import StructureFactor
from structure_factor.spatstat_interface import SpatstatInterface
from structure_factor.spatial_windows import UnitBoxWindow


# test the method  ``compute_scattering_intensity`` of the calss ``StructureFactor``

## loading data for test :
Poisson point process, Ginibre point pross, lattice $\mathbb{Z}^2$ matching processus of Michael Andreas Klatt, Günter Last, D. Yogeshwaran that we will denoted by kly defined in https://arxiv.org/abs/1810.00265

In [ ]:
#importing big data
import pickle
import rpy2.robjects as robjects
path_data = "../data"
with open(os.path.join(path_data, "data_ginibre.dat"), "rb") as data:
    n_ginibre, ginibre = pickle.load(data, encoding="bytes")

robjects.r["load"](os.path.join(path_data, "../data/data_kly.dat"))
data_R_2 = robjects.r["out_2"]
data_kly_all = np.array(data_R_2)

We load big set of data that we already sample for time perpose. As we know that the method ``compute_scattering_intensity``work only for point configurations in **cubic windows** so for a point processes folling in a random window we will only take the points falling in a cubic window .
For example a configuration of the Ginibre point process is always contained in a ball, so we will project it to a cubic window.
To compare the effect of **dismissing points** we will test the poisson point process ``data_poisson_disk`` sampled in a disk via the function ``poisson_in_disk``   then projected to a cubic window ``data_poisson_cube_2`` and a poisson point process ``data_poisson_cube`` sampled directely in a cubic window  via the function ``poisson_in_rectangle``.

## Please consider using code from `homogeneous_poisson_process.py` and `spatial_windows.py` as presented in `Poisson-process-and-windows.ipynb`

In [ ]:
def poisson_in_rectangle(xMin, xMax, yMin, yMax, lambda_p):
    xDelta = xMax - xMin;yDelta = yMax - yMin #rectangle dimensions
    areaTotal = xDelta * yDelta
    n_pois = scipy.stats.poisson(lambda_p * areaTotal).rvs()#Poisson number of points
    data_poisson_rectangle = np.zeros((n_pois, 2))
    xx = xDelta*scipy.stats.uniform.rvs(0, 1, ((n_pois, 1))) + xMin#x coordinates of Poisson points
    yy = yDelta*scipy.stats.uniform.rvs(0, 1, ((n_pois, 1))) + yMin#y coordinates of Poisson points
    data_poisson_rectangle[:,0] = xx[:,0]
    data_poisson_rectangle[:,1] = yy[:,0]
    return data_poisson_rectangle

In [ ]:
L_poisson=200
data_poisson_cube = poisson_in_rectangle(-100, 100, -100, 100, lambda_p=1)

In [ ]:
def poisson_in_disk(r, lambda_p):
    areaTotal = np.pi * r ** 2 #area of disk
    n_pois = np.random.poisson(lambda_p * areaTotal) #Poisson number of points
    data_poisson_disk = np.zeros((n_pois, 2))
    theta = 2 * np.pi * np.random.uniform(0, 1, n_pois) #angular coordinates 
    rho = r * np.sqrt(np.random.uniform(0, 1, n_pois))#radial coordinates 
    xx = rho * np.cos(theta)
    yy = rho * np.sin(theta)
    data_poisson_disk[:,0] = xx
    data_poisson_disk[:,1] = yy # rearrangement of poisson by increasing distance
    return data_poisson_disk

In [ ]:
raduis_poisson = 300
data_poisson_disk = poisson_in_disk(r=r_poisson, lambda_p=1)

In [ ]:
print("data_kly.shape", data_kly_all.shape)
print("data_poisson_disk.shape", data_poisson_disk.shape)
print("data_poisson_cube.shape", data_poisson_cube.shape)
print("data_ginibre.shape", ginibre.shape )

In [ ]:
#load big data in the corresponding shape :math: `n /times 2`
data_ginibre = np.array([np.array(np.real(ginibre)).reshape(-1,), np.array(np.imag(ginibre)).reshape(-1,)]).T
data_kly = np.array([data_kly_all[:,0], data_kly_all[:,1]]).T
data_z_2 = np.array([data_kly_all[:,2], data_kly_all[:,3]]).T

In [ ]:
#verifying the shape
print("data_kly.shape", data_kly.shape)
print("data_ginibre.shape", data_ginibre.shape )

In [ ]:
# truncating data_poisson and Ginibre in a cubic window
# poisson, and ginibre are contained in a ball while klatt and z_2 are in a rectangular window L = 300
L_poisson_2 = np.floor(raduis_poisson/np.sqrt(2))
index_poisson_in_cube = np.logical_and((np.abs(data_poisson_disk[:,0]) < (L_poisson_2/2)) , (np.abs(data_poisson_disk[:,1]) < (L_poisson_2/2)))
data_poisson_cube_2 = data_poisson_disk[index_poisson_in_cube]

raduis_ginibre = np.max(np.abs(ginibre))
L_ginibre = raduis_ginibre/np.sqrt(2)
index_ginibre_in_cube = np.logical_and((np.abs(data_ginibre[:,0]) < L_ginibre/2) , (np.abs(data_ginibre[:,1]) < L_ginibre/2))
data_ginibre_cube = data_ginibre[index_ginibre_in_cube]

print("data_poisson_cube.shape", data_poisson_cube_2.shape)
print("data_ginibre_cube.shape", data_ginibre.shape )
fig, ax = plt.subplots(2, 2, figsize=(9,6))
ax[0,0].plot(data_ginibre[:,0], data_ginibre[:,1], 'b,')
ax[0,0].title.set_text("data ginibre")
ax[0,1].plot(data_ginibre_cube[:,0], data_ginibre_cube[:,1], 'b,')
ax[0,1].title.set_text("data ginibre truncated in a cubic window")
ax[1,0].plot(data_poisson_disk[:,0], data_poisson_disk[:,1], 'b,')
ax[1,0].title.set_text("data poisson")
ax[1,1].plot(data_poisson_cube_2[:,0], data_poisson_cube_2[:,1], 'b,')
ax[1,1].title.set_text("data poisson truncated in a cubic window")

In [ ]:
sf_poisson_cube = StructureFactor(data_poisson_cube, intensity=1 )
sf_poisson_cube_2 = StructureFactor(data_poisson_cube_2, intensity=1 )
sf_ginibre_cube = StructureFactor(data_ginibre_cube, intensity = 1/np.pi )
sf_kly = StructureFactor(data_kly, intensity= 1)
sf_z_2 = StructureFactor(data_z_2, intensity=1)

# for Poisson point process
we know that the pair correlation function and the structure factor of the Poisson point process are equal to 1 so we always plot the line  $𝑦=1$  representing the théoretical values for a Poisson point process

In [ ]:
wave_length_poisson, scattering_intensity_poisson = sf_poisson_cube.compute_scattering_intensity(L=L_poisson,maximum_k =10,meshgrid_size=None)

the method ``plot_scattering_intensity`` take as input the output of the method ``compute_scattering_intensity`` to plot them.
also take as optional argument ``binning_parameter``correspnding to the parameters used for binning of ``si``, and ``exact_sf`` which the true sctructure factor function

In [ ]:
sf_poisson_cube.plot_scattering_intensity(wave_length_poisson.ravel(), scattering_intensity_poisson.ravel(), bins=40)

In [ ]:
wave_length_poisson, scattering_intensity_poisson = sf_poisson_cube.compute_scattering_intensity(L=L_poisson,maximum_k =10,meshgrid_size=150)

In [ ]:
sf_poisson_cube.plot_scattering_intensity(wave_length_poisson, scattering_intensity_poisson, plot_type="all",  bins=40, )

In [ ]:
wave_length_poisson_2, scattering_intensity_poisson_2 = sf_poisson_cube_2.compute_scattering_intensity(L=L_poisson_2,maximum_k =10,meshgrid_size=150)

In [ ]:
sf_poisson_cube_2.plot_scattering_intensity(wave_length_poisson_2, scattering_intensity_poisson_2, plot_type="all",  bins=40 )

# for Ginibre point process

In [ ]:
exact_sf_ginibre = lambda x : 1 - np.exp(-x**2/4) # exact structure factor for the Ginibre point process

In [ ]:
x=np.linspace(0,10, 1000)
plt.plot(x, exact_sf_ginibre(x), 'r')
plt.title("sf of Ginibre")
plt.show()

In [ ]:
norm_k_ginibre, scattering_intensity_ginibre = sf_ginibre_cube.compute_scattering_intensity(L=L_ginibre,maximum_k =20)


In [ ]:
sf_ginibre_cube.plot_scattering_intensity(norm_k_ginibre, scattering_intensity_ginibre, plot_type="plot", exact_sf= exact_sf_ginibre, bins=100 )

In [ ]:
x=np.linspace(0,25, 1000)
plt.plot(x, exact_sf_ginibre(x), 'r', label="exat sf ginibre ")
plt.plot(norm_k_ginibre, scattering_intensity_ginibre, 'k.')
plt.show()

In [ ]:
norm_k_ginibre_, scattering_intensity_ginibre_ = sf_ginibre_cube.compute_scattering_intensity(L=L_ginibre,maximum_k =10,meshgrid_size=300)

In [ ]:
sf_ginibre_cube.plot_scattering_intensity(norm_k_ginibre_, scattering_intensity_ginibre_, plot_type="all", bins=40 )

# for the processus of  Michael Andreas Klatt, Günter Last, D. Yogeshwaran that we will denoted by kly defined in https://arxiv.org/abs/1810.00265

In [ ]:
norm_k_kly, si_kly = sf_kly.compute_scattering_intensity(L=300, maximum_k=20)

In [ ]:
sf_kly.plot_scattering_intensity(norm_k_kly, si_kly, plot_type="plot", bins=40 )

In [ ]:
norm_k_kly_, si_kly_ = sf_kly.compute_scattering_intensity(L=300, maximum_k=10, meshgrid_size=150)

In [ ]:
sf_kly.plot_scattering_intensity(norm_k_kly_, si_kly_, plot_type="all", bins=40)

# for a lattice $\mathbb{Z}^2$

In [ ]:
norm_k_z2, si_z2 = sf_z_2.compute_scattering_intensity(L=300, maximum_k=50)

In [ ]:
sf_z_2.plot_scattering_intensity(norm_k_z2, si_z2, plot_type="plot", bins=40 )

In [ ]:
plt.plot(norm_k_z2, si_z2, 'b.')

In [ ]:
norm_k_z2_, si_z2_ = sf_z_2.compute_scattering_intensity(L=300, maximum_k=20, meshgrid_size=150)

In [ ]:
sf_z_2.plot_scattering_intensity(norm_k_z2_, si_z2_, plot_type="all", bins=40)